In [22]:
global t
t = 0

TERRAINS = {
    "W": 1,  
    "G": 1.2,
    "T": min(1.5, 0.1 * t),
    "M": 1.6,
}

In [23]:
def heuristic(a, b):
    """Manhattan distance multiplied by the lowest terrain cost (water)."""
    (x1, y1), (x2, y2) = a, b
    return (abs(x1 - x2) + abs(y1 - y2)) * TERRAINS["W"]

In [24]:
import os
import time

def print_grid(opened, closed, path):
    os.system('cls' if os.name == 'nt' else 'clear')  # Clear terminal
    print("\nGrid:")
    for r in range(ROWS):
        row_repr = ""
        for c in range(COLS):
            cell = grid[r][c]
            if (r, c) == start:
                row_repr += "S "
            elif (r, c) == goal:
                row_repr += "G "
            elif (r, c) in path:
                row_repr += "* "
            elif (r, c) in opened:
                row_repr += "O "
            elif (r, c) in closed:
                row_repr += "X "
            else:
                row_repr += cell + " "
        print(row_repr)
    
    print("\nOpened list:")
    print(opened)

    print("\nClosed list:")
    print(closed)

    print("\nCurrent path (partial):")
    print(path)

    time.sleep(0.05)  # Slow down for step-by-step visualization

# First Grid:

In [25]:
grid = [
    list("WGGTG"),
    list("GMWMM"),
    list("GTMWG"),
    list("GGGMM"),
    list("TMMGW"),
    list("GWGMG"),
    list("GGMTM"),
    list("WGGWM"),
]

ROWS, COLS = len(grid), len(grid[0])
start = (0, 0)  
goal = (6, 4)   

In [26]:
def neighbors(node):
    x, y = node
    for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        nx, ny = x + dx, y + dy
        if 0 <= nx < ROWS and 0 <= ny < COLS:
            if TERRAINS[grid[nx][ny]] is not None:
                yield (nx, ny)

In [27]:
import heapq

def a_star(start, goal):
    global t
    open_heap = []
    heapq.heappush(open_heap, (0, start))

    came_from = {}
    g_score = {start: 0}

    opened_order, closed_order = [], []

    while open_heap:
        current_f, current = heapq.heappop(open_heap)
        closed_order.append(current)
        t += 1

        if current == goal:                     
            break                               

        for n in neighbors(current):
            terrain_cost = TERRAINS[grid[n[0]][n[1]]]
            tentative_g = g_score[current] + terrain_cost

            if n not in g_score or tentative_g < g_score[n]:
                came_from[n] = current
                g_score[n] = tentative_g
                f = tentative_g + heuristic(n, goal)
                heapq.heappush(open_heap, (f, n))
                if n not in opened_order:
                    opened_order.append(n)

    path = []
    node = goal
    while node in came_from or node == start:
        path.append(node)
        if node == start:
            break
        node = came_from[node]
    path.reverse()

    total_cost = g_score.get(goal)
    return path, opened_order, closed_order, total_cost

In [28]:
import heapq

def a_star_visual(start, goal):
    open_set = []
    heapq.heappush(open_set, (0 + heuristic(start, goal), 0, start))
    
    came_from = {}
    g_score = {start: 0}
    opened = set()
    closed = set()
    
    while open_set:
        _, cost, current = heapq.heappop(open_set)
        
        if current in closed:
            continue
        
        closed.add(current)
        opened.discard(current)

        # Reconstruct partial path for visualization
        path = []
        temp = current
        while temp in came_from:
            path.append(temp)
            temp = came_from[temp]
        path = path[::-1]

        print_grid(opened, closed, path)

        if current == goal:
            print("\nGoal reached!")
            return path

        for neighbor in neighbors(current):
            if neighbor in closed:
                continue
            tentative_g = g_score[current] + 1
            if neighbor not in g_score or tentative_g < g_score[neighbor]:
                came_from[neighbor] = current
                g_score[neighbor] = tentative_g
                f_score = tentative_g + heuristic(neighbor, goal)
                heapq.heappush(open_set, (f_score, tentative_g, neighbor))
                opened.add(neighbor)

    print("No path found.")
    return []

# Run the algorithm
final_path = a_star_visual(start, goal)


Grid:
S G G T G 
G M W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
set()

Closed list:
{(0, 0)}

Current path (partial):
[]

Grid:
S * G T G 
O M W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0)}

Closed list:
{(0, 1), (0, 0)}

Current path (partial):
[(0, 1)]

Grid:
S X O T G 
* O W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 1), (0, 2)}

Closed list:
{(0, 1), (1, 0), (0, 0)}

Current path (partial):
[(1, 0)]

Grid:
S * * T G 
X O W M M 
O T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 1), (2, 0)}

Closed list:
{(0, 1), (1, 0), (0, 2), (0, 0)}

Current path (partial):
[(0, 1), (0, 2)]

Grid:
S * X O G 
X * O M M 
O T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 2), (0, 3), (2, 0)}

Closed list:
{(0, 1), (0, 0), (1, 1), (0, 2), (1, 0)}

Current path (partial):
[(0, 1), (1, 1)]

Grid:

In [29]:
path, opened, closed, cost = a_star(start, goal)

In [30]:
cost

10.2

In [31]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)



Grid:
S G G T G 
* M W M M 
* T M W G 
* G G M M 
* M M G W 
* * * * G 
G G M * G 
W G G W M 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (2, 0), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 3), (3, 3), (2, 2), (2, 4), (3, 4), (3, 0), (2, 1), (3, 1), (4, 1), (3, 2), (4, 2), (5, 1), (4, 0), (6, 1), (5, 0), (5, 2), (6, 2), (5, 3), (7, 1), (6, 0), (4, 3), (7, 0), (6, 3), (5, 4), (7, 3), (6, 4)]

Closed nodes (order processed):
[(0, 0), (0, 1), (1, 0), (0, 2), (0, 3), (0, 4), (1, 3), (2, 3), (1, 4), (2, 4), (1, 2), (2, 0), (2, 1), (3, 1), (3, 2), (2, 2), (4, 1), (5, 1), (5, 2), (6, 1), (3, 3), (4, 2), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (6, 0), (6, 1), (3, 3), (5, 3), (6, 3), (6, 4)]

Final path from S to G:
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (5, 1), (5, 2), (5, 3), (6, 3), (6, 4)]


# Inadmissible Heuristic

If you select, for example, the highest terrain cost as the multiplier for the heuristic, A* will overestimate the cost, leading to a loss of optimality

In [32]:
def heuristic(a, b):
    """Manhattan distance multiplied by the lowest terrain cost (water)."""
    (x1, y1), (x2, y2) = a, b
    return (abs(x1 - x2) + abs(y1 - y2)) * TERRAINS["M"]

In [33]:
path, opened, closed, cost = a_star(start, goal)

In [34]:
print("\nGrid:")
for r in range(ROWS):
    row_repr = ""
    for c in range(COLS):
        symbol = grid[r][c]
        if (r, c) == start:
            symbol = "S"
        elif (r, c) == goal:
            symbol = "G"
        elif (r, c) in path:
            symbol = "*"
        row_repr += symbol + " "
    print(row_repr)

print("\nOpened nodes (order added):")
print(opened)

print("\nClosed nodes (order processed):")
print(closed)

print("\nFinal path from S to G:")
print(path)


Grid:
S * * * * 
G M W M * 
G T M W * 
G G G M * 
T M M G * 
G W G M * 
G G M T G 
W G G W M 

Opened nodes (order added):
[(1, 0), (0, 1), (1, 1), (0, 2), (1, 2), (0, 3), (1, 3), (0, 4), (1, 4), (2, 4), (3, 4), (2, 3), (4, 4), (3, 3), (5, 4), (4, 3), (6, 4), (5, 3)]

Closed nodes (order processed):
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]

Final path from S to G:
[(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (6, 4)]


In [35]:
cost

11.799999999999999

In [36]:
a_star_visual(start, goal)


Grid:
S G G T G 
G M W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
set()

Closed list:
{(0, 0)}

Current path (partial):
[]

Grid:
S * G T G 
O M W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0)}

Closed list:
{(0, 1), (0, 0)}

Current path (partial):
[(0, 1)]

Grid:
S * * T G 
O O W M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0), (1, 1)}

Closed list:
{(0, 1), (0, 2), (0, 0)}

Current path (partial):
[(0, 1), (0, 2)]

Grid:
S * * * G 
O O O M M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 0), (1, 1), (1, 2)}

Closed list:
{(0, 1), (0, 2), (0, 3), (0, 0)}

Current path (partial):
[(0, 1), (0, 2), (0, 3)]

Grid:
S * * * * 
O O O O M 
G T M W G 
G G G M M 
T M M G W 
G W G M G 
G G M T G 
W G G W M 

Opened list:
{(1, 2), (1, 1), (1, 0), (1, 3)}

Closed list:
{(0, 1), (0, 4), (0, 0), (0, 3), (0, 2)}

Current path (p

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 4),
 (2, 4),
 (3, 4),
 (4, 4),
 (5, 4),
 (6, 4)]